In [2]:
import regex
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy
import pyLDAvis

In [3]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df = pd.read_csv('cleaned_US2021.csv')
df.head(10)

C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Unnamed: 0.1,title,score,id,url,coms_num,author,upvotes,word_count
0,0,0,st. paul ‘sovereign citizen’ records himself s...,1764,naxke1,https://minnesotareformer.com/2021/05/08/st-pa...,219,zardogo,1764,10
1,1,1,district keeps kids at home because of lack of...,73,naywl0,https://apnews.com/article/coronavirus-pandemi...,3,brashendeavors,73,11
2,2,2,tacoma man says he was stabbed after asking so...,2660,nab3az,https://www.kiro7.com/news/local/tacoma-man-sa...,599,DICK-FUCK-PUSSY-SUCK,2660,13
3,3,3,israeli air strike destroys gaza tower block,342,na47s3,https://www.aljazeera.com/news/2021/5/11/israe...,285,throwaway0152384,342,7
4,4,4,elon musk says tesla will stop accepting bitco...,4068,nb17cy,https://www.cnbc.com/2021/05/12/elon-musk-says...,860,FletchForPresident,4068,14
5,5,5,norman lloyd dies: ‘st. elsewhere’ actor whose...,96,na7yga,https://deadline.com/2021/05/norman-lloyd-dead...,3,DemiFiendRSA,96,15
6,6,6,ohio governor says state will give $1m to 5 ra...,434,nb2niw,https://www.wbrz.com/news/ohio-governor-says-s...,106,cyclinginvancouver,434,13
7,7,7,'i'm very rich': back seat tesla rider pulls s...,81,nb5pe9,http://ktvu.com/news/back-seat-tesla-rider-pul...,37,reef001,81,17
8,8,8,americans as young as 12 begin to get vaccinat...,115,najakn,https://us.cnn.com/2021/05/11/health/us-corona...,19,pcaversaccio,115,14
9,9,9,nato exercises sweep europe amid russian escal...,29,nb4bby,https://www.weny.com/story/43867503/nato-exerc...,6,marsianer,29,13


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data = df['title'].tolist()
data_words = list(sent_to_words(data))

print(data_words[0])

C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
['st', 'paul', 'sovereign', 'citizen', 'records', 'himself', 'stealing', 'vaccines', 'from', 'workers']


In [6]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)
print(bigram_mod[data_words[0]])

['st', 'paul_sovereign', 'citizen', 'records_himself', 'stealing_vaccines', 'from', 'workers']
C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
['st', 'citizen', 'record', 'stealing_vaccine', 'worker']


In [8]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]
C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 20 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[(0, '0.031*"police" + 0.016*"state" + 0.015*"covid" + 0.015*"aid" + 0.014*"dead" + 0.014*"fire" + 0.013*"rule" + 0.013*"arrest" + 0.012*"vaccine" + 0.012*"allege"'), (1, '0.024*"covid" + 0.021*"gaza" + 0.017*"vaccinated" + 0.017*"fire" + 0.015*"american" + 0.014*"death" + 0.014*"citizen" + 0.014*"get" + 0.014*"israel" + 0.014*"rocket"'), (2, '0.025*"charge" + 0.022*"say" + 0.020*"covid" + 0.020*"report" + 0.018*"fuel" + 0.018*"official" + 0.017*"man" + 0.013*"gas" + 0.013*"pipeline" + 0.011*"new"'), (3, '0.035*"pipeline" + 0.018*"colonial" + 0.018*"hack" + 0.016*"sex" + 0.013*"large" + 0.012*"israel" + 0.011*"old" + 0.011*"new" + 0.0

In [10]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.271075  0.141056       1        1  28.439726
1      0.139656 -0.277177       2        1  24.682376
0     -0.269716 -0.149351       3        1  23.589668
3     -0.141015  0.285472       4        1  23.288230, topic_info=         Term        Freq       Total Category  logprob  loglift
83     police   76.000000   76.000000  Default  30.0000  30.0000
194  pipeline  121.000000  121.000000  Default  29.0000  29.0000
232    charge   72.000000   72.000000  Default  28.0000  28.0000
20       gaza   54.000000   54.000000  Default  27.0000  27.0000
267    report   58.000000   58.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
69    convict   19.374936   25.220308   Topic4  -4.8166   1.1936
176       old   27.213376   54.549811   Topic4  -4.4768   0.7618
270    israel   28.284248   74.532559   Topic4  -4.4382   0.4883
49        new   26.798247   74.535510   Topic4  -4.4922   0.4343
11        man   23.930661   92.501768   Topic4  -4.6054   0.1052

[181 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
107       2  0.986179  ahmaud_arbery
138       2  0.027514            aid
138       3  0.963001            aid
17        2  0.950972            air
87        2  0.930289      airstrike
...     ...       ...            ...
159       3  0.959400          whale
143       1  0.519153          woman
143       3  0.375938          woman
143       4  0.107411          woman
4         2  0.936913         worker

[179 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])

In [11]:
df = pd.read_csv('cleaned_WN2021.csv')
df.head(10)

C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Unnamed: 0.1,title,score,id,url,coms_num,author,upvotes,word_count
0,0,0,Bosnian Serb ex-leader Karadzic serve genocide...,44,naup26,https://www.aljazeera.com/news/2021/5/12/bosni...,https://www.aljazeera.com/news/2021/5/12/bosni...,vokiel,44,11
1,1,1,Taiwan computer brand wiped mainland China’s o...,32,nb3z2v,https://www.scmp.com/tech/tech-trends/article/...,https://www.scmp.com/tech/tech-trends/article/...,ccpshrilldetective,32,11
2,2,2,"""A Hell Out Here"": COVID-19 Ravages Rural India",11,nbcuse,https://www.ndtv.com/india-news/a-hell-out-her...,https://www.ndtv.com/india-news/a-hell-out-her...,Revolutionary_Ant852,11,8
3,3,3,Federal budget 2021: Foreign aid cut national ...,3,nbafrh,https://www.news.com.au/finance/economy/federa...,https://www.news.com.au/finance/economy/federa...,kamikkazet,3,13
4,4,4,Weary Gaza marks Muslim feast violence spreads...,17,nbdzub,https://apnews.com/article/middle-east-israel-...,https://apnews.com/article/middle-east-israel-...,GDW312,17,10
5,5,5,"Bolsonaro's approval falls 24%, lowest ever, s...",59,nbfg0w,https://www.reuters.com/world/americas/bolsona...,https://www.reuters.com/world/americas/bolsona...,ThatsMySpatula,59,44
6,6,6,There 7m-13m excess deaths worldwide pandemic,23,nbhh5x,https://www.economist.com/briefing/2021/05/15/...,https://www.economist.com/briefing/2021/05/15/...,Kill-Bacon-Tea,23,10
7,7,7,Japanese towns drop plans host Olympic athlete...,31,nbbwnf,https://www.reuters.com/lifestyle/sports/olymp...,https://www.reuters.com/lifestyle/sports/olymp...,BenTom_,31,10
8,8,8,Brazilian police target indigenous leaders gov...,38,nbcwk6,https://www.theguardian.com/world/2021/may/13/...,https://www.theguardian.com/world/2021/may/13/...,PauloPatricio,38,8
9,9,9,"Climate emissions shrinking stratosphere, scie...",56,napocb,https://www.theguardian.com/environment/2021/m...,https://www.theguardian.com/environment/2021/m...,Akephalos_Agares,56,7


In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data = df['title'].tolist()
data_words = list(sent_to_words(data))

print(data_words[0])

['bosnian', 'serb', 'ex', 'leader', 'karadzic', 'serve', 'genocide', 'term', 'uk', 'jail']
C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)
print(bigram_mod[data_words[0]])

['bosnian', 'serb', 'ex', 'leader', 'karadzic', 'serve', 'genocide', 'term', 'uk', 'jail']
C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
['bosnian', 'serb', 'ex', 'leader', 'karadzic', 'serve', 'genocide', 'term', 'uk', 'jail']


In [15]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]
C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 20 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[(0, '0.022*"world" + 0.018*"report" + 0.017*"uk" + 0.017*"china" + 0.015*"covid" + 0.014*"import" + 0.014*"muslim" + 0.013*"accuse" + 0.013*"weary" + 0.013*"feast"'), (1, '0.022*"russia" + 0.021*"first" + 0.017*"china" + 0.017*"nuclear" + 0.016*"chernobyl" + 0.014*"find" + 0.013*"reaction" + 0.013*"spiking_inaccessible" + 0.013*"chamber" + 0.012*"law"'), (2, '0.031*"covid" + 0.024*"india" + 0.022*"case" + 0.018*"israel" + 0.018*"un" + 0.017*"record" + 0.015*"war" + 0.014*"follow" + 0.013*"fight" + 0.012*"palestinian"'), (3, '0.070*"israel" + 0.055*"gaza" + 0.036*"violence" + 0.032*"israeli" + 0.021*"death" + 0.021*"warn" + 0.020*"kil

In [17]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

C:\Users\0626t\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.283519 -0.184836       1        1  34.418515
2     -0.153518  0.265275       2        1  24.384952
0      0.192820 -0.236238       3        1  22.010670
1      0.244217  0.155799       4        1  19.185863, topic_info=            Term        Freq       Total Category  logprob  loglift
35        israel  310.000000  310.000000  Default  30.0000  30.0000
34          gaza  216.000000  216.000000  Default  29.0000  29.0000
39      violence  133.000000  133.000000  Default  28.0000  28.0000
120      israeli  131.000000  131.000000  Default  27.0000  27.0000
57         death   79.000000   79.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
191       record   20.315873   66.991725   Topic4  -4.6281   0.4578
186  coronavirus   14.901675   21.945546   Topic4  -4.9381   1.2639
21         india   16.676811   81.231071   Topic4  -4.8255   0.0677
171         case   15.531332   73.112486   Topic4  -4.8967   0.1019
173         long   14.909299   30.170653   Topic4  -4.9376   0.9461

[196 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
525       4  0.953908     abroad
330       3  0.979590     accuse
439       2  0.982860        act
316       3  0.960907     africa
288       1  0.984434  airstrike
...     ...       ...        ...
40        3  0.967297      weary
505       3  0.945909    western
942       1  0.990914   withdraw
196       3  0.872588      world
196       4  0.119767      world

[203 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2])